In [7]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 5

import sys
from datetime import datetime
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime

  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)


You are already connected to a glueetl session 8a37a81d-8688-47f4-8760-040cf73fca9f.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.


You are already connected to a glueetl session 8a37a81d-8688-47f4-8760-040cf73fca9f.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 3.0


You are already connected to a glueetl session 8a37a81d-8688-47f4-8760-040cf73fca9f.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session 8a37a81d-8688-47f4-8760-040cf73fca9f.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 5
Setting new number of workers to: 5



In [8]:
args = getResolvedOptions(sys.argv, ['input_file','output_file'])
date_str = datetime.today().strftime('%Y-%m-%d')
input_file = args['input_file']
output_file = f"s3://hit-details/output/{date_str}_SearchKeywordPerformance.tab"

GlueArgumentError: the following arguments are required: --input_file, --output_file


In [ ]:
hits_df=spark.read.format("csv").option("delimiter","\t").option("header", True).load(input_file)

def find_top_ref(df, hit_time_gmt, user_agent, ip, page_url):
    referrer_list = hits_df.filter((col("hit_time_gmt") <= hit_time_gmt) & (col("user_agent") == user_agent) & (col("ip") == ip) & (col("page_url") == page_url)).select("referrer").collect()
    referrer=""
    if len(referrer_list)>0:
        if len(referrer_list[0])>0:
            referrer=referrer_list[0][0]
    if referrer is not None:
        if 'esshopzilla' not in referrer:
            return referrer
        else:
            return find_top_ref(df,hit_time_gmt, user_agent, ip, referrer)
    return referrer

In [6]:
order_df = hits_df.filter("pagename=='Order Complete'")
result_dict = []
for order in order_df.collect():
    result=find_top_ref(hits_df, order['hit_time_gmt'], order['user_agent'], order['ip'], order['page_url'])
    if result and 'esshopzilla' not in result:
        url = result.split(".com")[0].split(".")[-1] + ".com"
        keyword = result.split("?")[-1].split("q=")[1].split("&")[0]
        revenue=int(order['product_list'].split(";")[2]) * int(order['product_list'].split(";")[3])
        result_dict.append({"Search Engine Domain": url, "Search Keyword":keyword, "Revenue": revenue})
final_df=sc.parallelize(result_dict).toDF()
final_df=final_df.groupBy("Search Engine Domain","Search Keyword").agg(sum("Revenue").alias("Revenue"))
final_df = final_df.coalesce(1)

In [ ]:
final_df.write.mode("overwrite").format("csv").option("delimiter", "\t").option("header", True).save(output_file)